* Daniel Morales 21785
* Brandon Sicay 21757
# Task 1
### Algoritmo AC-3 y su relación con el algoritmo de Backtracking Search
* El Algoritmo de Consistencia de Arcos 3 (AC-3) es un algoritmo clásico en el campo de la inteligencia artificial, específicamente en la resolución de problemas de satisfacción de        restricciones. Su objetivo es reducir el espacio de búsqueda de un problema CSP mediante la aplicación de una técnica conocida como "consistencia de arcos" antes de proceder con métodos de búsqueda más intensivos, como el backtracking.
* La relación entre AC-3 y el algoritmo de Backtracking Search es que AC-3 a menudo se utiliza como un paso de preprocesamiento antes de aplicar backtracking. Al eliminar valores inconsistentes de los dominios de las variables, AC-3 reduce el espacio de búsqueda, lo que puede hacer que el backtracking sea más eficiente al evitar la exploración de asignaciones que no llevarían a una solución. Además, durante el backtracking, se puede aplicar AC-3 después de cada asignación de variable para mantener la consistencia de arcos a lo largo de la búsqueda, lo cual puede ayudar a identificar más rápidamente las asignaciones que violarían las restricciones y, por lo tanto, a reducir el número de backtracks necesarios.

### Definición del término “Arc Consistency”
* Significa que para cualquier par de variables en un problema, cada opción posible para una variable tiene al menos una opción correspondiente en la otra que no rompe las reglas establecidas entre ellas. Es como asegurarse de que, no importa lo que elijas para una variable, siempre tendrás al menos una elección válida para la otra que mantenga todo en regla.

# Task 2
### Entendimiento de especificaciónes y restricciones 
* Tenemos cuatro estudiantes con un conjunto distribuido y posiblemente superpuesto de exámenes.
* Cada estudiante puede tener un número variable de exámenes, sin mínimo ni máximo definido.
* Los exámenes deben programarse en tres días específicos: lunes, martes y miércoles.
* Las restricciones son:
    * Ningún estudiante puede tener más de un examen por día.
    * No pueden haber dos exámenes del mismo curso para diferentes estudiantes programados el mismo día.

In [33]:
import time

In [43]:
# Datos de entrada
estudiantes = ['Estudiante1', 'Estudiante2', 'Estudiante3', 'Estudiante4']
examenes = ['Matemáticas', 'Historia', 'Ciencias', 'Inglés', 'Arte', 'Educación Física', 'Informática']
dias = ['Lunes', 'Martes', 'Miércoles']

# Asignar exámenes
asignacion_inicial = {
    'Estudiante1': ['Matemáticas', 'Arte','Historia'],
    'Estudiante2': ['Inglés', 'Arte'],
    'Estudiante3': ['Educación Física', 'Informática','Arte'], 
    'Estudiante4': ['Ciencias','Matemáticas']
}



In [44]:
asignaciones = {estudiante: {} for estudiante in estudiantes}
for estudiante, exams in asignacion_inicial.items():
    for exam in exams:
        asignaciones[estudiante][exam] = None  

def es_asignacion_valida(asignacion, estudiante, examen, dia):
    if any(dia == asignacion[e].get(examen) for e in asignacion):
        return False
    if dia in asignacion[estudiante].values():
        return False
    return True

def backtracking(asignacion, estudiantes, examenes, dias, estudiante_idx=0):
    if all(asignacion[est].get(exam) is not None for est in asignacion for exam in asignacion[est]):
        return asignacion  

    estudiante_actual = estudiantes[estudiante_idx]

    for examen, dia_asignado in asignacion[estudiante_actual].items():
        if dia_asignado is None:  
            for dia in dias:
                if es_asignacion_valida(asignacion, estudiante_actual, examen, dia):
                    asignacion[estudiante_actual][examen] = dia  
                    if backtracking(asignacion, estudiantes, examenes, dias, estudiante_idx) is not None:
                        return asignacion
                    asignacion[estudiante_actual][examen] = None 

    if estudiante_idx + 1 < len(estudiantes):
        return backtracking(asignacion, estudiantes, examenes, dias, estudiante_idx + 1)
    
    return None  
start_time = time.time()
resultado = backtracking(asignaciones, estudiantes, examenes, dias)
end_time = time.time()

if resultado:
    for estudiante, exam_dias in resultado.items():
        print(f"{estudiante}: {exam_dias}")
    duration = end_time - start_time
    print("Solución encontrada en ",duration, "segundos")
else:
    print("No se encontró una solución válida.")


Estudiante1: {'Matemáticas': 'Lunes', 'Arte': 'Martes', 'Historia': 'Miércoles'}
Estudiante2: {'Inglés': 'Lunes', 'Arte': 'Miércoles'}
Estudiante3: {'Educación Física': 'Martes', 'Informática': 'Miércoles', 'Arte': 'Lunes'}
Estudiante4: {'Ciencias': 'Lunes', 'Matemáticas': 'Martes'}
Solución encontrada en  0.0 segundos


In [39]:
asignacion_inicial = {
    'Estudiante1': {'Matemáticas': None, 'Arte': None, 'Historia': None},
    'Estudiante2': {'Inglés': None, 'Arte': None},
    'Estudiante3': {'Educación Física': None, 'Informática': None, 'Arte': None}, 
    'Estudiante4': {'Ciencias': None, 'Matemáticas': None}
}
beam_width = 29  # Con un beam_width de 29 es el valor mínimo con el que el algoritmo logra encontrar una solución.

def es_asignacion_completa(asignacion):
    for exams in asignacion.values():
        if None in exams.values():
            return False
    return True

def es_asignacion_valida(asignacion):
    examenes_por_dia = {}
    examenes_estudiante_por_dia = {est: {} for est in asignacion}
    
    for est, exams in asignacion.items():
        for exam, dia in exams.items():
            if dia:  
                if dia in examenes_estudiante_por_dia[est]:
                    return False
                examenes_estudiante_por_dia[est][dia] = exam
                
                if exam not in examenes_por_dia:
                    examenes_por_dia[exam] = set()
                if dia in examenes_por_dia[exam]:
                    return False
                examenes_por_dia[exam].add(dia)
    return True

def generar_posibles_asignaciones(asignacion):
    nuevas_asignaciones = []
    for estudiante, examenes in asignacion.items():
        for examen, dia in examenes.items():
            if dia is None:  
                for nuevo_dia in dias:
                    nueva_asignacion = {e: exams.copy() for e, exams in asignacion.items()}
                    nueva_asignacion[estudiante][examen] = nuevo_dia
                    if es_asignacion_valida(nueva_asignacion):
                        nuevas_asignaciones.append(nueva_asignacion)
                break  
    return nuevas_asignaciones

def evaluar_asignacion(asignacion):
    total_asignados = sum(dia is not None for exams in asignacion.values() for dia in exams.values())
    dias_usados = {dia for exams in asignacion.values() for dia in exams.values() if dia}
    return total_asignados + len(dias_usados)  

def beam_search(asignacion_inicial, beam_width):
    posibles_asignaciones = [asignacion_inicial]
    
    while posibles_asignaciones:
        todas_posibles = []
        for asignacion in posibles_asignaciones:
            todas_posibles.extend(generar_posibles_asignaciones(asignacion))
        
        todas_posibles.sort(key=evaluar_asignacion, reverse=True)
        posibles_asignaciones = todas_posibles[:beam_width]
        
        for asignacion in posibles_asignaciones:
            if es_asignacion_completa(asignacion) and es_asignacion_valida(asignacion):
                return asignacion
    
    return None  
start_time = time.time()
resultado = beam_search(asignacion_inicial, beam_width)
end_time = time.time()
if resultado:
    print("Solución encontrada:")
    for estudiante, exam_dias in resultado.items():
        print(f"{estudiante}: {exam_dias}")
    duration = end_time - start_time
    print("Solución encontrada en ",duration, "segundos")
else:
    print("No se encontró una solución.")


Solución encontrada:
Estudiante1: {'Matemáticas': 'Lunes', 'Arte': 'Martes', 'Historia': 'Miércoles'}
Estudiante2: {'Inglés': 'Lunes', 'Arte': 'Miércoles'}
Estudiante3: {'Educación Física': 'Martes', 'Informática': 'Miércoles', 'Arte': 'Lunes'}
Estudiante4: {'Ciencias': 'Lunes', 'Matemáticas': 'Martes'}
Solución encontrada en  0.022996902465820312 segundos


In [38]:
import random

estudiantes_examen = {
    'Estudiante1': ['Matemáticas', 'Arte', 'Historia'],
    'Estudiante2': ['Inglés', 'Arte'],
    'Estudiante3': ['Educación Física', 'Informática', 'Arte'], 
    'Estudiante4': ['Ciencias', 'Matemáticas']
}
estudiantes = ['Estudiante1', 'Estudiante2', 'Estudiante3', 'Estudiante4']
examenes = ['Matemáticas', 'Historia', 'Ciencias', 'Inglés', 'Arte', 'Educación Física', 'Informática']
dias = ['Lunes', 'Martes', 'Miércoles']

def generar_asignacion_inicial(estudiantes_examen):
    asignacion = {}
    for estudiante, examenes in estudiantes_examen.items():
        asignacion[estudiante] = {}
        for examen in examenes:
            asignacion[estudiante][examen] = None  
    return asignacion

def es_asignacion_valida(asignacion):
    examenes_por_dia = {dia: set() for dia in dias}
    for estudiante, exam_dias in asignacion.items():
        dias_estudiante = set()  
        for examen, dia in exam_dias.items():
            if dia is None:
                continue
            if examen in examenes_por_dia[dia] or dia in dias_estudiante:
                return False
            examenes_por_dia[dia].add(examen)
            dias_estudiante.add(dia)
    return True

def local_search(estudiantes_examen, iteraciones=1000):
    asignacion_inicial = generar_asignacion_inicial(estudiantes_examen)
    asignacion_actual = asignacion_inicial
    for i in range(iteraciones):
        estudiante_aleatorio = random.choice(list(asignacion_actual.keys()))
        examen_aleatorio = random.choice(list(asignacion_actual[estudiante_aleatorio].keys()))
        nuevo_dia = random.choice(dias)

        nueva_asignacion = {e: exams.copy() for e, exams in asignacion_actual.items()}
        nueva_asignacion[estudiante_aleatorio][examen_aleatorio] = nuevo_dia
        
        if es_asignacion_valida(nueva_asignacion):
            asignacion_actual = nueva_asignacion
            
    return asignacion_actual

start_time = time.time()
solucion = local_search(estudiantes_examen)
end_time = time.time()
print("Solución encontrada:")
if solucion:
    for estudiante, exam_dias in solucion.items():
        print(f"{estudiante}: {exam_dias}")
    duration = end_time - start_time
    print("Solución encontrada en ",duration, "segundos")
else:
    print("No se encontró una solución.")

Solución encontrada:
Estudiante1: {'Matemáticas': 'Martes', 'Arte': 'Miércoles', 'Historia': 'Lunes'}
Estudiante2: {'Inglés': 'Lunes', 'Arte': 'Martes'}
Estudiante3: {'Educación Física': 'Miércoles', 'Informática': 'Martes', 'Arte': 'Lunes'}
Estudiante4: {'Ciencias': 'Miércoles', 'Matemáticas': 'Lunes'}
Solución encontrada en  0.009337663650512695 segundos


### Comparación de Tiempo de Ejecución:

* Backtracking es el más rápido, con un tiempo casi instantáneo. Esto podría deberse a que el problema tiene un espacio de soluciones relativamente pequeño, o el algoritmo puede descartar rápidamente ramas no prometedoras del espacio de búsqueda.
* Beam Search toma más tiempo que los otros dos algoritmos. A pesar de encontrar la misma solución que el Backtracking, su mayor tiempo de ejecución se debe probablemente a la exploración de un número más amplio de posibles soluciones antes de converger.
* Local Search muestra un rendimiento intermedio en términos de tiempo. Aunque es más rápido que el Beam Search, es un poco más lento que el Backtracking, posiblemente debido a la generación aleatoria de soluciones vecinas y la evaluación de su validez.

### Comparación de Soluciones:

* Tanto el Backtracking como el Beam Search llegaron a la misma solución, lo que indica que, para este problema y conjunto de datos específico, ambos algoritmos son capaces de encontrar una solución óptima eficientemente.
* Local Search, por otro lado, encontró una solución diferente. Esto no necesariamente significa que sea una solución de menor calidad; simplemente es otra forma válida de asignar los exámenes bajo las restricciones dadas. La búsqueda local puede encontrar soluciones alternativas dependiendo del estado inicial y las transiciones entre estados.

### Conclusión:

* En términos de eficiencia temporal, el Backtracking es superior para este problema específico.
* Sin embargo, el Beam Search y la Búsqueda Local también encuentran soluciones válidas, con la Búsqueda Local ofreciendo una alternativa que podría ser útil en casos donde la diversidad de soluciones es deseable.
* La elección del algoritmo depende del contexto específico: si se busca la solución más rápidamente posible, el Backtracking parece ser la mejor opción. Si se prefiere explorar un espacio de soluciones más amplio, incluso a costa de un tiempo de ejecución ligeramente mayor, Beam Search y Búsqueda Local son alternativas válidas.